In [1]:
# Imports
import numpy as np
import pandas as pd

In [24]:
# Load CNN/DailyMail data (small set)
#data = np.load('../data/cnn_dailymail.npz', allow_pickle=True)
#data = np.load('../data/cnn_dailymail_small.npz', allow_pickle=True)

In [23]:
# Load NewsAPI web-scraped data (small set)
data = np.load('../data/news_api_data_small.npz', allow_pickle=True)

# Initialize splits
X_train = data['X_train'] # Articles
y_train = data['y_train'] # Highlights
X_test = data['X_test']
y_test = data['y_test']
X_val = data['X_val']
y_val = data['y_val']

print(X_train) # X_train is a numpy array of Articles

['"However, we do believe that there has been a pattern of behaviour that falls below the standards expected of MPs and individuals looking after donations to local campaign funds which lie outside the direct jurisdiction of the Conservative party."'
 'The navy said as there were no other aircraft nor vessels in nearby waters, involvement of another country in the incident was unlikely.'
 'Getty Images Copyright: Getty Images This file photo shows Sudanese migrants sitting near a centre run by MSF in Calais Image caption: This file photo shows Sudanese migrants sitting near a centre run by MSF in Calais\n\nThe boss of the UK arm of Médecins Sans Frontières, also known as Doctors Without Borders, has criticised the UK\'s Rwanda policy - saying "there is no evidence" it will act as a deterrent and stop the boats (something ministers have previously said).\n\nNatalie Roberts describes people making the journey across the English Channel as "some of the most marginalised people on Earth", 

In [26]:
# Print one article from X_train
print(X_train[1]) # this one is more of a sentence than an article
print(X_train[3]) # this is a full article

The navy said as there were no other aircraft nor vessels in nearby waters, involvement of another country in the incident was unlikely.
Manchester United manager Erik ten Hag, speaking to ITV1: "It was an incredible game, a strange game too. We had total control for so long and then gave it away in the last part of the game. We did show resilience to win the penalty shootout.

"We had control for 75 minutes and have to give Coventry a compliment for what they did in the last 15 minutes.

"We have to improve. We talk a lot about this. First you have to put yourself in a winning position but then you have to take it over the line. In these moments we are making mistakes and not keeping our responsibilities."

On whether there is a problem with mentality: "We can't say this group doesn't have the right mentality because they showed character today, but in the final part there was a lack of discipline.

"We have to keep the ball, keep making passes, go for goal and then also be strong in 

In [27]:
# Create dataframe from nparrays
df1 = pd.DataFrame(
    {'article': X_train,
     'highlights': y_train
    })
df1.head()

,article,highlights
0,"""However, we do believe that there has been a ...",The Fylde MP will leave Parliament at the next...
1,The navy said as there were no other aircraft ...,Defence Minister Minoru Kihara said a thorough...
2,Getty Images Copyright: Getty Images This file...,The prime minister has vowed to keep the Commo...
3,"Manchester United manager Erik ten Hag, speaki...",Listen to BBC radio commentary & follow live t...
4,"Mr Khan said: ""I'm hoping that reassures the L...",An independent police force will review the ca...


In [29]:
df1.shape

(6, 2)

In [30]:
df1.describe()

,article,highlights
count,6,6
unique,6,6
top,"""However, we do believe that there has been a ...",The Fylde MP will leave Parliament at the next...
freq,1,1
